# PlGreen Maskell 2017
implementing the third algorithm second is just a simplificaiton of the first that uses less memory so may be useful for barkla but is kind of trivial at the smae time

* this extension uses smc for the resampling alg as well, by the sounds of it
* still on static param estimaiton

In [ ]:
%%capture output
%pip install numpy
%pip install matplotlib

In [22]:
import numpy as np
import matplotlib.pyplot as plt

beta_0 = 1
beta_e = 100
mu_0 = 0

def normal_dist(x,mu,var):
    sigma = var**(1/2)
    return (1/(sigma*(np.pi*2)**(1/2)))*np.e**(-0.5*((x-mu)/sigma)**2)

def yt_func(x_t:float) -> float:
    # this is the real theta that we don't know
    theta = 1
    return theta*x_t

def zt_func(x_t:float) -> float:
    error = np.random.randn(1)[0]*(1/beta_e)
    return yt_func(x_t)+error

def prob_z_given_theta(theta, z_1_to_t, x_1_to_t):
    """
    The equation says its proportional hence the k but i
    don't know what the k is i assume it is cancelled
    when you compute weights, just added var k to 
    reflec the proportional relationship have no idea
    if k matters though
    """
    k = 1
    sig_sum = 0
    for zt, xt in zip(z_1_to_t,x_1_to_t):
        sig_sum += (zt-yt_func(xt))**2
    return k*np.e**((-beta_e/2)*sig_sum)

def prob_theta(theta):
    return np.e**((-beta_0/2)*(theta-mu_0)**2)

def beta_func(x_1_to_t):
    sig_sum = 0
    for xt in x_1_to_t:
        sig_sum += xt**2
    return beta_e*sig_sum+beta_0

def mu_func(z_1_to_t,x_1_to_t):
    sig_sum = 0
    for zt, xt in zip(z_1_to_t,x_1_to_t):
        sig_sum += xt*zt
    beta = beta_func(x_1_to_t)
    return (1/beta)*(beta_e*sig_sum+beta_0*mu_0)

def prob_theta_given_z(theta,z_1_to_t,x_1_to_t):
    return np.e**((-beta_func(x_1_to_t)/2)*theta-mu_func(z_1_to_t,x_1_to_t))

# just some wrapper functions might help my understanding
def liklihood(z_1_to_t,x_1_to_t):
    return prob_z_given_theta(z_1_to_t,x_1_to_t)

def prior(theta):
    return prob_theta(theta)

# im really not sure about this function
# this being wrong does also make my l kernel wrong
def prior_conditional(theta_hat,theta):
    return normal_dist(theta_hat,theta,1)

# since we are using gaussian proposal dist we acc don't need this
# since this and the term above return identical p so cancel
def l_kernel(theta, theta_hat):
    return  normal_dist(theta,theta_hat,1)


def sample_prior(theta_arr=None,n=100):
    if theta_arr is None:
        theta_arr = [mu_0]*n
    return theta_arr+(np.random.randn(n)*beta_0)

def posterior(z_1_to_t,x_1_to_t):
    return prob_theta_given_z(z_1_to_t,x_1_to_t)

def get_initial_weights(z_arr,x_arr,theta):
    return [liklihood(z_arr, x_arr, t) for t in theta]

def norm_weights(weights):
    return weights/sum(weights)

def reset_weights(n):
    return [1]*n

# this should just take zt the whole liklihood thing is probs 
# wrong
def new_weights(weights,z_arr,theta,theta_hat):
    w_hat = []
    for w,th,th_hat in zip(weights,theta,theta_hat):
        temp1 = liklihood(z_arr,th_hat)/liklihood(z_arr,th)
        temp2 = l_kernel(th, th_hat)/prior_conditional(th_hat,th)
        w_hat.append(w*temp1/temp2)
    return w_hat

def get_neff(weights):
    squared_sum = 0
    for w in weights:
        squared_sum += w*w
    return 1/squared_sum

def resample(curr_theta, weights):
    new_theta = []
    new_theta = np.random.choice(curr_theta,size = len(curr_theta), p=weights)
    return np.array(new_theta)

total_t = 500
# remember the observations happen sequentially
xt_arr = []
xt_arr.append(np.random.rand(1)[0])
# grab all of yt bear in mind i shouldn't know this it will
# only be needed for plotting later
yt_arr = np.array([yt_func(x) for x in xt_arr])

zt_arr = []
n = 100

theta = sample_prior()
zt_arr.append(zt_func(xt_arr[-1]))
weights = get_initial_weights(zt_arr)

time = 1
while time<total_t:
    # new system change
    xt_arr.append(np.random.rand(1)[0])
    # new observation is then made
    zt_arr.append(zt_func(xt_arr[-1]))

    weights = norm_weights()
    neff = get_neff(weights)
    while (neff<n/2):
        resample(theta,weights)
        weights = reset_weights(n)
        theta_hat = sample_prior(theta,n)
        weights = new_weights(weights,zt_arr,theta,theta_hat)
        theta = theta_hat
        weights = norm_weights(weights)
        neff = get_neff(weights)
    time+=1
    weights = new_weights(weights,zt_arr,theta)

yt_arr = np.array([yt_func(x) for x in xt_arr])

# basically there is something wrong with the liklihood as the
# function i am using and alg 3 are pretty incompatible, so any call
# to ne weigths is definitely wrong
# other stuff is ofc also wrong tho but, i suppose for 
# known xt yt is conditional on just theta, so maybe we are meant
# to predict yt_hat like it says and then send that into our
# liklihood as it holds the info for both x and theta at the 
# same time so it would make sense for our conditional 
# statement idk though

TypeError: get_initial_weights() missing 2 required positional arguments: 'x_arr' and 'theta'